
# Closing the Gap: Interactive Analysis
### Mastercard Data Challenge 2025 - Team Tornadoes

**Goal**: Demonstrate how we use data-driven insights to identify, measure, and close the Inclusive Growth Score (IGS) gap between Census Tract 105 and Tract 1100 in Talladega County, AL.

This notebook walks through:
1. **The Data**: Exploring our comprehensive dataset and feature engineering.
2. **Unsupervised Learning**: Clustering tracts to find peers.
3. **The Problem**: Visualizing the 27-point IGS gap.
4. **The Gaps**: Identifying key pillars (Digital, Housing, Entrepreneurship).
5. **The Prediction**: Using our ML model to forecast the impact of interventions.


In [ ]:

# Install dependencies in the current kernel
%pip install pandas numpy matplotlib seaborn scikit-learn joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Setup paths relative to submission/Notebooks/
DATA_DIR = os.path.join("..", "Input_Datasets")
CODE_DIR = os.path.join("..", "Code")

# Add code to path for model import
sys.path.append(CODE_DIR)

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")



## 1. Load the Data
We load our comprehensive dataset which includes historical metrics for all census tracts.


In [ ]:

# Robust Data Loading: Find the dataset file automatically
possible_paths = [
    # Prioritize the full dataset if available (for detailed charts)
    os.path.join("..", "Input_Datasets", "unified_ml_dataset.csv"),
    "unified_ml_dataset.csv",
    
    # Fallback to the submission file (summary data)
    "final_submission.csv",
    "Sample_submission.csv",
    os.path.join("..", "final_submission.csv"),
    os.path.join("..", "Sample_submission.csv"),
]

data_path = None
for p in possible_paths:
    if os.path.exists(p):
        data_path = p
        break

# Fallback: search for similar names if renamed
if data_path is None:
    # Search current directory (.) and parent (..)
    search_dirs = [".", "..", os.path.join("..", "Input_Datasets")]
    for d in search_dirs:
        if os.path.exists(d):
            try:
                for f in os.listdir(d):
                    # Looser matching: looks for csv with 'submission' or 'unified' or 'final'
                    if f.endswith('.csv') and ('unified' in f.lower() or 'submission' in f.lower() or 'final' in f.lower()):
                        data_path = os.path.join(d, f)
                        break
            except:
                pass # Ignore permission errors
        if data_path: break

if data_path is None:
    # Check if running on Colab
    if os.getcwd() == "/content":
        print("[WARNING] You appear to be running on Google Colab.")
        print("[ACTION] Please drag and drop 'final_submission.csv' into the Files pane on the left.")
    else:
        print(f"[WARNING] Dataset not found. Current working directory: {os.getcwd()}")
        print(f"   Looking for 'final_submission.csv' in: {possible_paths}")
        
    raise FileNotFoundError("Could not find the dataset CSV file. Please check the file location.")

print(f"[OK] Loading data from: {data_path}")
df = pd.read_csv(data_path)

# Filter for our target tracts
tract_105 = df[df['tract_fips'] == 1121010500].sort_values('Year')
tract_1100 = df[df['tract_fips'] == 1121011100].sort_values('Year')

print(f"Tract 105 Data Points: {len(tract_105)}")
print(f"Tract 1100 Data Points: {len(tract_1100)}")

# Show available columns
cols = ['Year', 'Inclusive Growth Score']
extra_cols = ['median_household_income', 'poverty_rate']
for c in extra_cols:
    if c in df.columns:
        cols.append(c)
        
tract_105[cols].tail()



## 2. Data Cleaning & Feature Engineering
Before modeling, we enriched the raw data with derived metrics to capture the full economic picture.


In [ ]:

# Example of feature engineering logic used in our pipeline
# We calculated ratios to normalize comparisons across tracts of different sizes

def calculate_derived_features(data):
    # Housing Cost Burden (Rent > 30% of income)
    if 'rent_30_35_percent_income' in data.columns and 'total_renter_households' in data.columns:
        burdened = (data['rent_30_35_percent_income'] + 
                   data['rent_35_40_percent_income'] + 
                   data['rent_40_50_percent_income'] + 
                   data['rent_50_plus_percent_income'])
        data['housing_burden_rate'] = (burdened / data['total_renter_households']) * 100
    
    # Poverty Rate
    if 'income_below_poverty' in data.columns and 'total_pop_poverty_status' in data.columns:
        data['poverty_rate'] = (data['income_below_poverty'] / data['total_pop_poverty_status']) * 100
        
    return data

# Display calculated features for Tract 105 (latest year)
if 'housing_cost_burden_rate' in df.columns:
    latest = df[df['tract_fips'] == 1121010500].iloc[-1]
    print(f"Derived Metrics for Tract 105 (2024):")
    print(f"  - Housing Cost Burden: {latest.get('housing_cost_burden_rate', 'N/A'):.1f}%")
    print(f"  - Poverty Rate: {latest.get('poverty_rate', 'N/A'):.1f}%")
    print(f"  - Unemployment Rate: {latest.get('unemployment_rate', 'N/A'):.1f}%")
else:
    print("Derived features not present in this dataset view.")



## 3. Unsupervised Learning: Clustering Analysis
To objectively categorize census tracts, we applied **K-Means Clustering** on the Inclusive Growth Score components (Growth vs. Inclusion). This allows us to group tracts with similar economic profiles without bias.


In [ ]:

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Prepare data for clustering (using latest year for all tracts)
latest_data = df.sort_values('Year').groupby('tract_fips').last().reset_index()

# Features for clustering
cluster_features = ['Growth', 'Inclusion', 'Inclusive Growth Score']
X = latest_data[cluster_features].dropna()

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-Means Clustering (k=3)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
latest_data['Cluster'] = kmeans.fit_predict(X_scaled)

# Visualize
plt.figure(figsize=(10, 6))
sns.scatterplot(data=latest_data, x='Inclusion', y='Growth', hue='Cluster', 
                palette='viridis', s=200, style='Cluster', markers=['o', 's', 'D'])

# Annotate Tract 105 and 1100
t105 = latest_data[latest_data['tract_fips'] == 1121010500].iloc[0]
t1100 = latest_data[latest_data['tract_fips'] == 1121011100].iloc[0]

plt.annotate('Tract 105\n(Target)', (t105['Inclusion'], t105['Growth']), 
             xytext=(t105['Inclusion']+2, t105['Growth']-5),
             arrowprops=dict(facecolor='red', shrink=0.05))

plt.annotate('Tract 1100\n(Benchmark)', (t1100['Inclusion'], t1100['Growth']), 
             xytext=(t1100['Inclusion']-10, t1100['Growth']+5),
             arrowprops=dict(facecolor='green', shrink=0.05))

plt.title('K-Means Clustering of Census Tracts (Growth vs Inclusion)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

print("Cluster interpretation:")
print(latest_data.groupby('Cluster')[['Inclusive Growth Score', 'Growth', 'Inclusion']].mean().sort_values('Inclusive Growth Score'))



## 4. Visualizing the Historical Gap (2017-2024)
Tract 105 has stagnated for 8 years, while Tract 1100 (just 10 miles away) has grown.


In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(tract_105['Year'], tract_105['Inclusive Growth Score'], 
         marker='o', linewidth=3, label='Tract 105 (Target)', color='#ef4444')
plt.plot(tract_1100['Year'], tract_1100['Inclusive Growth Score'], 
         marker='o', linewidth=3, label='Tract 1100 (Benchmark)', color='#10b981')

# Add threshold line
plt.axhline(y=45, color='gray', linestyle='--', alpha=0.5, label='Financial Inclusion Threshold (45)')

plt.title('The Inequality Gap: 8-Year IGS Trend', fontsize=14, fontweight='bold')
plt.ylabel('Inclusive Growth Score')
plt.ylim(0, 60)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()



## 5. Identifying Strategic Gaps
Why is there a gap? We compared key pillar metrics from the latest available year.


In [ ]:

# Compare specific metrics
metrics = {
    'Internet Access Score': 'Digital',
    'Affordable Housing Score': 'Housing',
    'Minority/Women Owned Businesses Score': 'Entrepreneurship',
    'Labor Market Engagement Index Score': 'Workforce'
}

# Check if we have these metrics
missing_metrics = [m for m in metrics.keys() if m not in df.columns]
if missing_metrics:
    print(f"[WARNING] Detailed pillar metrics not found in this dataset (using summary timeline file).")
    print("Skipping pillar breakdown chart.")
else:
    comparison_data = []
    latest_105 = tract_105.iloc[-1]
    latest_1100 = tract_1100.iloc[-1]

    for metric, label in metrics.items():
        comparison_data.append({
            'Pillar': label,
            'Tract': '105',
            'Score': latest_105[metric]
        })
        comparison_data.append({
            'Pillar': label,
            'Tract': '1100',
            'Score': latest_1100[metric]
        })

    df_comp = pd.DataFrame(comparison_data)

    plt.figure(figsize=(10, 5))
    sns.barplot(data=df_comp, x='Pillar', y='Score', hue='Tract', palette=['#ef4444', '#10b981'])
    plt.title('Strategic Pillar Scores: Where 105 Lags Behind', fontsize=14, fontweight='bold')
    plt.ylabel('Score (0-100)')
    plt.ylim(0, 100)
    plt.show()



## 6. Predictive Modeling: Forecasting Impact (2025-2029)
We use our constrained ML model (`ConstrainedIGSPredictor`) to forecast future scores under two scenarios:
1. **Baseline**: No new interventions.
2. **Full Intervention**: Investing in Digital, Housing, Entrepreneurship, and Workforce.


In [ ]:

try:
    from ml.models.constrained_predictor import ConstrainedIGSPredictor
    
    # Load trained model
    predictor = ConstrainedIGSPredictor()
    predictor.load() # Loads from ../Code/ml/models/igs_constrained_model.joblib
    
    tract_id = "1121010500"
    years_ahead = 5
    years = list(range(2025, 2030))
    
    # Predict Baseline
    baseline_preds = predictor.predict(tract_id, [], years_ahead)
    
    # Predict Full Intervention
    # We apply a "Conservative Implementation Factor" of 0.3 to the raw model lift
    # This accounts for real-world adoption friction and budget constraints.
    raw_intervention_preds = predictor.predict(
        tract_id, 
        ['digital', 'housing', 'entrepreneurship', 'workforce'], 
        years_ahead
    )
    
    baseline_arr = np.array(baseline_preds)
    raw_lift = np.array(raw_intervention_preds) - baseline_arr
    
    # Scale lift to be conservative (+5.1 pts instead of theoretical +17)
    conservative_lift = raw_lift * 0.3
    intervention_preds = list(np.round(baseline_arr + conservative_lift, 1))
    
    # Plot
    plt.figure(figsize=(10, 6))
    
    # Historical context
    plt.plot(tract_105['Year'], tract_105['Inclusive Growth Score'], 
             color='#ef4444', alpha=0.5, label='Historical')
    
    # Projections
    plt.plot(years, baseline_preds, marker='o', linestyle='--', 
             color='gray', label='Baseline Projection')
    plt.plot(years, intervention_preds, marker='o', linewidth=3, 
             color='#3b82f6', label='With Interventions')
    
    # Connect historical to projected
    plt.plot([2024, 2025], [tract_105['Inclusive Growth Score'].iloc[-1], baseline_preds[0]], 
             color='gray', linestyle='--', alpha=0.5)
    plt.plot([2024, 2025], [tract_105['Inclusive Growth Score'].iloc[-1], intervention_preds[0]], 
             color='#3b82f6', linestyle='--', alpha=0.5)
    
    plt.title('5-Year Impact Projection: Closing the Gap', fontsize=14, fontweight='bold')
    plt.axhline(y=45, color='gold', linestyle='-', alpha=0.8, label='Target Threshold (45)')
    plt.xlabel('Year')
    plt.ylabel('Projected IGS')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Projected 2029 Score (Baseline): {baseline_preds[-1]:.1f}")
    print(f"Projected 2029 Score (Intervention): {intervention_preds[-1]:.1f}")
    print(f"Net Lift: +{intervention_preds[-1] - baseline_preds[-1]:.1f} points")

except Exception as e:
    print(f"Could not run model predictions: {e}")
    print("Make sure you are running this from the submission/Notebooks directory and dependencies are installed.")



## 7. Conclusion
Our analysis shows that targeted investments in **Digital Infrastructure**, **Housing Affordability**, and **Entrepreneurship** can lift Census Tract 105 above the critical 45-point financial inclusion threshold within 3 years.

The data confirms that while the gap is large (27 points), it is closable with evidence-based interventions replicated from the successful benchmark Tract 1100.
